In [ ]:
# Mount to drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from natsort import natsorted
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import necessary files
train_folder_path = "/content/gdrive/MyDrive/BA865/data/train_frame/"
test_folder_path = "/content/gdrive/MyDrive/BA865/data/test_frame/"
train_label_path = '/content/gdrive/MyDrive/BA865/data/train.csv'
test_label_path = '/content/gdrive/MyDrive/BA865/data/test.csv'

df_train = pd.read_csv(train_label_path)
df_test = pd.read_csv(test_label_path)

def create_list(path_folder):
  data = []
  for image_name in natsorted(os.listdir(path_folder)):
    image_path = os.path.join(path_folder, image_name)
    # Create dictonary for dataframe input
    data.append({
        'Path': image_path,
        'Image_name': image_name,
        'Video_name': image_name.split('-')[0] +'.avi',
    })
  return data

In [ ]:
# Create df_train_final
df_train_final = pd.DataFrame(create_list(train_folder_path))
# Encode the labels
label = LabelEncoder().fit_transform(df_train['Label'])
df_train['Label_encode'] = label
# Merge dataframe with label
df_train_final = pd.merge(df_train_final, df_train[['Name','Label','Label_encode']], left_on='Video_name', right_on = 'Name')
df_train_final = df_train_final[['Path','Label','Label_encode']]

# Create df_test_final
df_test_final = pd.DataFrame(create_list(test_folder_path))
# Merge frame dataframe with label dataframe
df_test_final = pd.merge(df_test_final, df_test[['Name','Label']], left_on='Video_name', right_on = 'Name')
df_test_final = df_test_final[['Path','Label']]
df_test_final = pd.merge(df_test_final,df_train[['Label','Label_encode']],on='Label',how='left')

# Finalize dataset
df_train_final = df_train_final[['Path','Label_encode']]
df_test_final = df_test_final[['Path','Label_encode']]
df_train_final['Label_encode'] = df_train_final['Label_encode'].astype(str)
df_test_final['Label_encode'] = df_test_final['Label_encode'].astype(str)

In [ ]:
# Create train datagen
train_datagen = ImageDataGenerator(
          rescale=1./255,
          rotation_range=45,
          shear_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')
# Create test datagen
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train generator - not using WandB
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train_final,
    directory=None,
    x_col='Path',
    y_col='Label_encode',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
)

# Create test generator - not using WandB
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test_final,
    directory=None,
    x_col='Path',
    y_col='Label_encode',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
)

# Get number of classes: for classification
num_classes = df_train_final['Label_encode'].nunique()

Found 4724 validated image filenames belonging to 20 classes.
Found 5764 validated image filenames belonging to 20 classes.
